In [2]:
from ray.experimental.serve.frontend import ZMQClient
from multiprocessing import Process

In [5]:
zmq_client = ZMQClient(port=8082)

In [ ]:
def noop():
    return zmq_client.query(1.0, 1000, actor="NoOp")

In [ ]:
latencies = []
for i in range(1,1601): 
    now = time.perf_counter()
    for _ in range(i):
        p1 = Process(target=noop)
        p1.start()
    p1.join()
    after = time.perf_counter()
    latencies += [after - now]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1,1601), latencies, 'o-')
plt.title("Latency vs. Number of Requests using ZMQ FrontEnd")
plt.xlabel("Number of Queries")
plt.ylabel("Latency in ns")